In [1]:
"""Derived portfolios

Shows how to use derived portfolios, a type of portfolio that inherits the contents from a parent portfolio.

Attributes
----------
transactions
holdings
derived portfolios
"""

'Derived portfolios\n\nShows how to use derived portfolios, a type of portfolio that inherits the contents from a parent portfolio.\n\nAttributes\n----------\ntransactions\nholdings\nderived portfolios\n'

# Derived portfolios

This notebook demonstrates the LUSID [derived portfolios](https://support.finbourne.com/what-is-a-derived-portfolio). A derived portfolio is a portfolio which inherits the contents (or is "derived") from another parent portfolio. The derived portfolio also contains the entire `transaction` and/or `holding` history of the parent portfolio. You can then modify the data in the derived portfolio without impacting the parent portfolio.

In the example below, we will demonstrate the following workflow:

<ul> (1) Create a parent UK Equity portfolio with some FTSE 100 stocks </ul> 
<ol> (2) Derive a new portfolio from the parent </ol> 
<ol> (3) Cancel a transaction in the derived portfolio but not the parent </ol> 
<ol> (4) Verify that the newly cancelled transaction updates the derived portfolio but not the parent portfolio holdings </ol>

### Setup LUSID

In [2]:
# Import general purpose packages
import os
import json
from datetime import datetime, timedelta
import pytz

# Import lusid specific packages
import lusid
import lusid.models as models
from lusid.exceptions import ApiException
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id

# Import data wrangling packages
import pandas as pd

pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

Load a mapping file for DataFrame headers for the `build transaction` and `get holdings` response.

In [3]:
with open(r"config/build_transactions_mapping.json") as mappings_file:
    build_transactions_json_mapping = json.load(mappings_file)

with open(r"config/get_holdings_mapping.json") as mappings_file:
    get_holdings_json_mapping = json.load(mappings_file)

### 1) Load default transactions into a new scope

In [4]:
# Create a new scope

scope = "notebook-derived-portfolios"
portfolio_code = "EQUITY-UK" + "-" + create_scope_id()

In [5]:
# Load a file of equity transactions

transactions_file = r"data/derived/equity_transactions.csv"
transactions_df = pd.read_csv(transactions_file)
transactions_df["portfolio_code"] = portfolio_code

In [6]:
# Load portfolios, instruments, and transactions

seed_data_response = seed_data(
    api_factory,
    ["portfolios", "instruments", "transactions"],
    scope,
    transactions_df,
    "DataFrame",
)

In [7]:
# Define the transaction portfolio API

transaction_portfolio_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
derived_portfolios_api = api_factory.build(lusid.api.DerivedTransactionPortfoliosApi)

### 2) Lets check our holdings

We have:

* 300,000 units in Barclays

In [8]:
response = transaction_portfolio_api.get_holdings(
    scope=scope, code=portfolio_code, property_keys=["Instrument/default/Name"]
)

holdings_df = lusid_response_to_data_frame(
    response, rename_properties=True, column_name_mapping=get_holdings_json_mapping
)

holdings_df

,LusidInstrumentId,SubHoldingKeys,InstrumentName,SourcePortfolioId,SourcePortfolioScope(default-Properties),HoldingType,Units,SettledUnits,Amount-Cost,Currency-Cost,Amount-CostPortfolioCcy,Currenct-CostPortfolioCcy
0,LUID_YN3FEUD5,{},Aviva,EQUITY-UK-3967-b0b1-cb3a-22,notebook-derived-portfolios,P,132000.0,132000.0,660000.0,GBP,0.0,GBP
1,LUID_7EQ664NC,{},BHP,EQUITY-UK-3967-b0b1-cb3a-22,notebook-derived-portfolios,P,120000.0,120000.0,2160000.0,GBP,0.0,GBP
2,LUID_F87JY45S,{},Barclays,EQUITY-UK-3967-b0b1-cb3a-22,notebook-derived-portfolios,P,300000.0,300000.0,600000.0,GBP,0.0,GBP
3,LUID_8X5TTJTL,{},BP,EQUITY-UK-3967-b0b1-cb3a-22,notebook-derived-portfolios,P,200000.0,200000.0,1000000.0,GBP,0.0,GBP
4,LUID_H9EN8GYB,{},HSBC,EQUITY-UK-3967-b0b1-cb3a-22,notebook-derived-portfolios,P,40000.0,40000.0,240000.0,GBP,0.0,GBP
5,CCY_GBP,{},GBP,EQUITY-UK-3967-b0b1-cb3a-22,notebook-derived-portfolios,B,3260000.0,3260000.0,3260000.0,GBP,0.0,GBP
6,LUID_OSXJYVK6,{},Morrisons,EQUITY-UK-3967-b0b1-cb3a-22,notebook-derived-portfolios,P,360000.0,360000.0,720000.0,GBP,0.0,GBP
7,LUID_6RGYR2L4,{},Tesco,EQUITY-UK-3967-b0b1-cb3a-22,notebook-derived-portfolios,P,12000.0,12000.0,100000.0,GBP,0.0,GBP
8,LUID_2IXG0MUX,{},Rightmove,EQUITY-UK-3967-b0b1-cb3a-22,notebook-derived-portfolios,P,160000.0,160000.0,960000.0,GBP,0.0,GBP
9,LUID_OSHLMH9J,{},vodafone,EQUITY-UK-3967-b0b1-cb3a-22,notebook-derived-portfolios,P,900000.0,900000.0,900000.0,GBP,0.0,GBP


### 3) What transactions make up our Barclays holdings?

In [9]:
build_transactions_response = transaction_portfolio_api.build_transactions(
    scope=scope,
    code=portfolio_code,
    transaction_query_parameters=models.TransactionQueryParameters(
        start_date="2020-01-01", end_date="2020-12-31"
    ),
    property_keys=["Instrument/default/Name"],
)


build_transactions_df = lusid_response_to_data_frame(
    build_transactions_response,
    rename_properties=True,
    column_name_mapping=build_transactions_json_mapping,
)
build_transactions_df.query("InstrumentName == 'Barclays'")

,TransactionId,TransactionType,TransactionTypeDesc,ClientId,LusidInstrumentId,TransactionDate,SettlementDate,Units,TransactionAmount,Price,PriceType,TotalConsideration,TotalConsiderationCurrency,ExchangeRate,TransactionToPortfolioRate,TransactionCurrency,strategy(notebook-derived-portfolios-Properties),SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),ResultantHolding,InstrumentName,TransactionStatus,EntryDateTime,RealisedGainLoss,InstrumentCurrency
4,trd_0005,Buy,Purchase,EQ_1236,LUID_F87JY45S,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,150000.0,300000.0,2.0,Price,300000.0,GBP,1.0,0.0,GBP,ftse_tracker,EQUITY-UK-3967-b0b1-cb3a-22,notebook-derived-portfolios,150000.0,Barclays,Active,2021-03-15 11:37:08.554418+00:00,[],NaN
5,trd_0006,Buy,Purchase,EQ_1236,LUID_F87JY45S,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,150000.0,300000.0,2.0,Price,300000.0,GBP,1.0,0.0,GBP,ftse_tracker,EQUITY-UK-3967-b0b1-cb3a-22,notebook-derived-portfolios,300000.0,Barclays,Active,2021-03-15 11:37:08.554418+00:00,[],NaN


### 4) Create a derived portfolio

In this section we create a derived portfolio from our parent portfolio. The key message here - all the `transaction` history is inherited from the parent portfolio.

In [10]:
# Define a scope to hold the derived portfolio

new_scope = "TempReporting" + "-" + scope

print(f"The scope we'll use for the derived portfolios: {new_scope}")

The scope we'll use for the derived portfolios: TempReporting-notebook-derived-portfolios


In [11]:
# Create the derived portfolio

try:
    
    derived_portfolios_api.create_derived_portfolio(scope=new_scope,
                                               create_derived_transaction_portfolio_request = models.CreateDerivedTransactionPortfolioRequest(
                                                   display_name=portfolio_code,
                                                    description="Reporting portfolio",
                                                    code=portfolio_code,
                                                    parent_portfolio_id=models.ResourceId(scope=scope, code=portfolio_code),
                                                    created="2020-01-01",
                                                    corporate_action_source_id=None,
                                                    accounting_method=None,
                                                    sub_holding_keys=None,
                                               ))
    
except ApiException as e:
    print(json.loads(e.body)["name"])
    print(json.loads(e.body)["title"])

### 5) Cancel one of the Barclays transactions in the derived portfolio only

In [12]:
# Cancel one of the transactions with the CancelTransactions endpoint

cancel_response = transaction_portfolio_api.cancel_transactions(
    scope=new_scope, code=portfolio_code, transaction_ids=["trd_0006"]
)

### 6) Check holdings on the primary and the derived portfolio

As expected, we can see:

* The parent portfolio has 300,000 units of Barclays
* The derived portfolio has 150,000 units of Barclays

Run <i>GetHoldings</i> on the parent portfolio

In [13]:
response = transaction_portfolio_api.get_holdings(
    scope=scope, code=portfolio_code, property_keys=["Instrument/default/Name"]
)

holdings_df = lusid_response_to_data_frame(
    response, rename_properties=True, column_name_mapping=get_holdings_json_mapping
)

holdings_df.query("InstrumentName == 'Barclays'")

,LusidInstrumentId,SubHoldingKeys,InstrumentName,SourcePortfolioId,SourcePortfolioScope(default-Properties),HoldingType,Units,SettledUnits,Amount-Cost,Currency-Cost,Amount-CostPortfolioCcy,Currenct-CostPortfolioCcy
2,LUID_F87JY45S,{},Barclays,EQUITY-UK-3967-b0b1-cb3a-22,notebook-derived-portfolios,P,300000.0,300000.0,600000.0,GBP,0.0,GBP


Run <i>GetHoldings</i> on the derived portfolio

In [14]:
response = transaction_portfolio_api.get_holdings(
    scope=new_scope, code=portfolio_code, property_keys=["Instrument/default/Name"]
)

holdings_df = lusid_response_to_data_frame(
    response, rename_properties=True, column_name_mapping=get_holdings_json_mapping
)

holdings_df.query("InstrumentName == 'Barclays'")

,LusidInstrumentId,SubHoldingKeys,InstrumentName,SourcePortfolioId,SourcePortfolioScope(default-Properties),HoldingType,Units,SettledUnits,Amount-Cost,Currency-Cost,Amount-CostPortfolioCcy,Currenct-CostPortfolioCcy
2,LUID_F87JY45S,{},Barclays,EQUITY-UK-3967-b0b1-cb3a-22,TempReporting-notebook-derived-portfolios,P,150000.0,150000.0,300000.0,GBP,0.0,GBP
